In [1]:
import numpy as np
from torch import nn,optim
from torch.autograd import Variable
import torch
import sympy

In [55]:
primelist=[]
for i in range(1,1001):
    primelist.append(sympy.prime(i))
x1=torch.tensor(primelist)[:,np.newaxis]
y1=torch.ones(1000,1)
print(np.shape(x1),np.shape(y1))

torch.Size([1000, 1]) torch.Size([1000, 1])


In [56]:
nonprimelist=[]
for i in range(1,100000):
    if not sympy.isprime(i):
        nonprimelist.append(i)
    if len(nonprimelist)==1000:
        break
x2=torch.tensor(nonprimelist)[:,np.newaxis]
y2=torch.zeros(1000,1)
print(np.shape(x2),np.shape(y2))

torch.Size([1000, 1]) torch.Size([1000, 1])


In [57]:
x=torch.cat((x1,x2),dim=0).type(torch.FloatTensor)
y=torch.cat((y1,y2),dim=0).squeeze().long()
x,y=Variable(x),Variable(y)

In [83]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.layer1=nn.Sequential(nn.Linear(1,64),nn.Dropout(p=0.1),nn.LeakyReLU())
        self.layer2=nn.Sequential(nn.Linear(64,64),nn.Dropout(p=0.1),nn.LeakyReLU())
        self.layer3=nn.Sequential(nn.Linear(64,2),nn.LeakyReLU())
    def forward(self,x):
        #x=x.view(x.size()[0],-1)
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        return x
net=Net()
print(net)

Net(
  (layer1): Sequential(
    (0): Linear(in_features=1, out_features=64, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (layer2): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (layer3): Sequential(
    (0): Linear(in_features=64, out_features=2, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
)


In [94]:
optimizer=optim.Adam(net.parameters(),lr=1e-2)
loss_func=torch.nn.CrossEntropyLoss()

for i in range(10001):
    out=net(x)
    loss=loss_func(out,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i%400==0:
        prediction=torch.max(nn.functional.softmax(out,dim=1),1)[1]
        pred_y=prediction.data.numpy().squeeze()
        target_y=y.data.numpy()
        auc=(sum(pred_y==target_y))/2000
        print('epoch: {}, loss: {:4f}, auc: {:.4f}'.format(i,loss,auc))

epoch: 0, loss: 0.267576, auc: 0.9010
epoch: 400, loss: 0.287963, auc: 0.8980
epoch: 800, loss: 0.270863, auc: 0.9005
epoch: 1200, loss: 0.269221, auc: 0.9010
epoch: 1600, loss: 0.269241, auc: 0.9000
epoch: 2000, loss: 0.270819, auc: 0.9015
epoch: 2400, loss: 0.270131, auc: 0.9005
epoch: 2800, loss: 0.269252, auc: 0.9005
epoch: 3200, loss: 0.270378, auc: 0.9010
epoch: 3600, loss: 0.268619, auc: 0.9000
epoch: 4000, loss: 0.270105, auc: 0.9005
epoch: 4400, loss: 0.271660, auc: 0.9005
epoch: 4800, loss: 0.266402, auc: 0.9030
epoch: 5200, loss: 0.269174, auc: 0.8990
epoch: 5600, loss: 0.273309, auc: 0.9005
epoch: 6000, loss: 0.266994, auc: 0.9005
epoch: 6400, loss: 0.267055, auc: 0.9035
epoch: 6800, loss: 0.264930, auc: 0.9025
epoch: 7200, loss: 0.268502, auc: 0.8995
epoch: 7600, loss: 0.269742, auc: 0.8985
epoch: 8000, loss: 0.268952, auc: 0.9020
epoch: 8400, loss: 0.269285, auc: 0.9010
epoch: 8800, loss: 0.267064, auc: 0.9005
epoch: 9200, loss: 0.268374, auc: 0.9015
epoch: 9600, loss: 0.